In [1]:
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime

In [5]:
confirmed = pd.read_csv('C:/Users/maja1/OneDrive/Dokumenty/COVID-19-master/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series//time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('C:/Users/maja1/OneDrive/Dokumenty/COVID-19-master/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series//time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('C:/Users/maja1/OneDrive/Dokumenty/COVID-19-master/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series//time_series_covid19_recovered_global.csv')

print(confirmed.shape)
print(deaths.shape)
print(recovered.shape)

confirmed.head()

(264, 86)
(264, 86)
(250, 86)


Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
1            NaN        Albania  41.1533  20.1683        0        0        0   
2            NaN        Algeria  28.0339   1.6596        0        0        0   
3            NaN        Andorra  42.5063   1.5218        0        0        0   
4            NaN         Angola -11.2027  17.8739        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  4/3/20  4/4/20  4/5/20  4/6/20  4/7/20  \
0        0        0        0  ...     281     299     349     367     423   
1        0        0        0  ...     304     333     361     377     383   
2        0        0        0  ...    1171    1251    1320    1423    1468   
3        0        0        0  ...     439     466     501     525     545   
4        0        0        0  ...       8      10      14      16      17   

   4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
0     444     484      521      555      607  
1     400     409      416      433      446  
2    1572    1666     1761     1825     1914  
3     564     583      601      601      638  
4      19      19       19       19       19  

[5 rows x 86 columns]

In [7]:

confirmed2_transform = pd.melt(confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
deaths2_transform = pd.melt(deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
recovered_transform = pd.melt(recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])


print( confirmed2_transform.shape)
print(deaths2_transform.shape)
print( recovered_transform.shape)


confirmed2_transform.head()

(21648, 6)
(21648, 6)
(20500, 6)


Province/State Country/Region      Lat     Long     Date  value
0            NaN    Afghanistan  33.0000  65.0000  1/22/20      0
1            NaN        Albania  41.1533  20.1683  1/22/20      0
2            NaN        Algeria  28.0339   1.6596  1/22/20      0
3            NaN        Andorra  42.5063   1.5218  1/22/20      0
4            NaN         Angola -11.2027  17.8739  1/22/20      0

In [11]:
confirmed2_transform['Date'] = pd.to_datetime(confirmed2_transform['Date'])
deaths2_transform['Date'] = pd.to_datetime(deaths2_transform['Date'])
recovered_transform['Date'] = pd.to_datetime(recovered_transform['Date'])


confirmed2_transform.columns = confirmed2_transform.columns.str.replace('value', 'Confirmed')
deaths2_transform.columns = deaths2_transform.columns.str.replace('value', 'Deaths')
recovered_transform.columns = recovered_transform.columns.str.replace('value', 'Recovered')

In [13]:
confirmed2_transform.isnull().sum()

Province/State    14924
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
dtype: int64

In [14]:
deaths2_transform.isnull().sum()

Province/State    14924
Country/Region        0
Lat                   0
Long                  0
Date                  0
Deaths                0
dtype: int64

In [16]:
recovered_transform.isnull().sum()

Province/State    15006
Country/Region        0
Lat                   0
Long                  0
Date                  0
Recovered             0
dtype: int64

In [17]:
confirmed2_transform['Province/State'].fillna(confirmed2_transform['Country/Region'], inplace=True)
deaths2_transform['Province/State'].fillna(deaths2_transform['Country/Region'], inplace=True)
recovered_transform['Province/State'].fillna(recovered_transform['Country/Region'], inplace=True)

confirmed2_transform.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
dtype: int64

In [18]:
# Confirmed with Deaths
full_join = confirmed2_transform.merge(deaths2_transform[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print(full_join.shape)

# full join with Recovered
full_join = full_join.merge(recovered_transform[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print(full_join.shape)

full_join.head()

(21648, 7)
Shape of second join:  (21648, 8)


Province/State Country/Region      Lat     Long       Date  Confirmed  \
0    Afghanistan    Afghanistan  33.0000  65.0000 2020-01-22          0   
1        Albania        Albania  41.1533  20.1683 2020-01-22          0   
2        Algeria        Algeria  28.0339   1.6596 2020-01-22          0   
3        Andorra        Andorra  42.5063   1.5218 2020-01-22          0   
4         Angola         Angola -11.2027  17.8739 2020-01-22          0   

   Deaths  Recovered  
0       0        0.0  
1       0        0.0  
2       0        0.0  
3       0        0.0  
4       0        0.0

In [19]:
full_join.isnull().sum()

Province/State       0
Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
Deaths               0
Recovered         1230
dtype: int64

In [21]:
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

full_join.head()

Province/State Country/Region      Lat     Long       Date  Confirmed  \
0    Afghanistan    Afghanistan  33.0000  65.0000 2020-01-22          0   
1        Albania        Albania  41.1533  20.1683 2020-01-22          0   
2        Algeria        Algeria  28.0339   1.6596 2020-01-22          0   
3        Andorra        Andorra  42.5063   1.5218 2020-01-22          0   
4         Angola         Angola -11.2027  17.8739 2020-01-22          0   

   Deaths  Recovered Month-Year  
0       0        0.0   Jan-2020  
1       0        0.0   Jan-2020  
2       0        0.0   Jan-2020  
3       0        0.0   Jan-2020  
4       0        0.0   Jan-2020